In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd
import plotly.express as px

# Import your CRUD module
from animal_shelter import AnimalShelter

##############################
# Database Access via CRUD
##############################
username = "aacuser"
password = "CrossCrusaders707"
shelter = AnimalShelter(username, password)

# Default dataset fetch
df = pd.DataFrame.from_records(shelter.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Encode Grazioso Salvare logo
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.H1('SNHU CS-340 Dashboard - Shawn Henly')),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'}),
    html.Hr(),

    # Radio buttons for filtering rescue type
    html.Div([
        html.Label('Select Rescue Type'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'block'}
        )
    ]),

    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],
        page_action='native',
        page_current=0,
        page_size=10,
        filter_action='native',
        sort_action='native',
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'minWidth': '100px', 'width': '150px', 'maxWidth': '200px'}
    ),

    html.Hr(),

    # Charts and Map side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Controller / Callback Logic
#############################################

@app.callback(Output('datatable-id', 'data'),
              Input('filter-type', 'value'))
def update_dashboard(filter_type):
    if filter_type == 'Water Rescue':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "age_upon_outcome_in_weeks": {"$lte": 104}}
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "age_upon_outcome_in_weeks": {"$lte": 104}}
    elif filter_type == 'Disaster or Individual Tracking':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Doberman Pinscher", "Golden Retriever", "Bloodhound"]}, "age_upon_outcome_in_weeks": {"$lte": 104}}
    else:
        query = {}

    result_df = pd.DataFrame.from_records(shelter.read(query))
    if '_id' in result_df.columns:
        result_df.drop(columns=['_id'], inplace=True)
    return result_df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    Input('datatable-id', "derived_virtual_data"))
def update_graph(viewData):
    if viewData is None or len(viewData) == 0:
        return html.P("No data to display.")

    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Distribution of Breeds')
    return dcc.Graph(figure=fig)

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if not viewData:
        return html.P("No data to display on map.")
    dff = pd.DataFrame.from_dict(viewData)

    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    try:
        lat = dff.iloc[row]['location_lat']
        lon = dff.iloc[row]['location_long']
        breed = dff.iloc[row]['breed']
        name = dff.iloc[row]['name']
    except:
        return html.P("Invalid data for map display.")

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([html.H1("Animal Name"), html.P(name)])
            ])
        ])
    ]

# Run the dashboard on dash in Jupyter Notebook
app.run_server(debug="True")

Dash app running on http://127.0.0.1:13140/
